In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import pandas as pd
from env import github_token, github_username

import prepare
import acquire
import explore
#import model

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filter="ignore"

from bs4 import BeautifulSoup
from mergedeep import merge

NameError: name 'train' is not defined

<hr style="border-top: 10px groove lightblue; margin-top: 1px; margin-bottom: 1px"></hr>

# `Acquire Data`
## Using  [Acquire.py](https://github.com/bert-jason-ray/nlp-group-project/blob/main/acquire.py) file to bring in new dataframe.`

In [ ]:
# define the acquired file as new data frame
df = acquire.get_github_data()
df

<hr style="border-top: 10px groove lightblue; margin-top: 1px; margin-bottom: 1px"></hr>

# `Preapare Data`

## Using  [Prepare.py](https://github.com/bert-jason-ray/nlp-group-project/blob/main/prepare.py) to summon Clean, Stemmed, and Lemmatized data.

In [ ]:
df = prepare.prep_github_data(df,column = 'readme_contents', extra_words=[], exclude_words=['musicbot'])
df

## Looking at the amounts of times a specific language is used.

In [ ]:
df = df.drop(columns = ['readme_contents', 'stemmed','clean'])

In [ ]:
df = df.dropna()

In [ ]:
df

<hr style="border-top: 10px groove lightblue; margin-top: 1px; margin-bottom: 1px"></hr>

# `Splitting Data`

def split_github_data(df):
    '''
    This function performs split on github data, stratify language.
    Returns train, validate, and test dfs.
    '''
    train, test = train_test_split(df, test_size=.2, 
                                        random_state=123, stratify=df.language)
    #train, validate = train_test_split(train_validate, test_size=.3, 
                                   #random_state=123, stratify=train_validate.language)

    print('train--->', train.shape)
    #print('validate--->', validate.shape)
    print('test--->', test.shape)
    return train, test



In [ ]:
train, test = prepare.split_github_data(df)

<hr style="border-top: 10px groove lightblue; margin-top: 1px; margin-bottom: 1px"></hr>

# `EXPLORE`

In [ ]:
# Starting our exploration we quickly noticed a trend of a handful of programing languages dominating our data.
# As a result, we made the decision to focus on these main languages in an effort build a model more accurately discern between them
def programing_language_distribution():
    sns.set_theme(style="white")
    ax = sns.countplot(x="language", data=train, palette="Set3",order = train['language'].value_counts().index)
    ax.tick_params(axis='x', rotation=90)
    ax.set_xlabel('Programming Language (Readme.md)', size = 16)
    ax.set_ylabel('Count / Frequency', size = 16)
    ax.set_title("Programming Language Distribution", size = 20)
    plt.show()
programing_language_distribution()

### Takeaways:
 - **JavaScript & Python seem to dominate, while Java & TypeScript and all others seem to be at the 10 range.**

In [ ]:
train.language.value_counts()

<hr style="border-top: 10px groove lightblue; margin-top: 1px; margin-bottom: 1px"></hr>

## `Possible questions`**:**
- What are key words in all `READMEs`?
- Are there commom words in `READMEs` that all `programming languages` use?
- Do words in `READMEs` vary based on `programming languages` used?
- Do lenght of words vary in `READMEs` based on `programming languages`?

In [ ]:
# Set up word counts dataframe
all_text = ' '.join(train.lemmatized)
javascript_text = ' '.join(train[train.language == 'JavaScript'].lemmatized)
python_text = ' '.join(train[train.language == 'Python'].lemmatized)
typescript_text = ' '.join(train[train.language == 'TypeScript'].lemmatized)
java_text = ' '.join(train[train.language == 'Java'].lemmatized)
go_text = ' '.join(train[train.language == 'Go'].lemmatized)
kotlin_text = ' '.join(train[train.language == 'Kotlin'].lemmatized)

In [ ]:
def all_top_words():
    all_freq = pd.Series(str(all_text).split()).value_counts()
    javascript_freq = pd.Series(str(javascript_text).split()).value_counts()
    python_freq = pd.Series(str(python_text).split()).value_counts()
    typeScript_freq = pd.Series(str(typescript_text).split()).value_counts()
    java_freq = pd.Series(str(java_text).split()).value_counts()
    go_freq = pd.Series(str(go_text).split()).value_counts()
    kotlin_freq = pd.Series(str(kotlin_text).split()).value_counts()
    word_counts = pd.concat([all_freq, javascript_freq, python_freq, typeScript_freq,java_freq, go_freq, kotlin_freq], sort=True, axis=1)
    word_counts.columns = ['all', 'JavaScript', 'python', 'typescript','java', 'go', 'kotlin']
    word_counts = word_counts.fillna(0).apply(lambda s: s.astype(int))
    top_30 = word_counts.sort_values(by='all', ascending=False).head(30)
    return top_30
all_top_words()

<hr style="border-top: 10px groove lightblue; margin-top: 1px; margin-bottom: 1px"></hr>

# `What are key words in all READMEs & Are there commom words in READMEs that all programming languages use?`

In [ ]:
def word_distribution_vizual():
    # Visualize word distribution
    word_counts.sort_values(by='all', ascending=False)[['all','JavaScript', 'python', 'typescript','java', 'go', 'kotlin']].head(9).plot.bar()
    plt.title('Words Used Most in README Files')
    plt.style.use('seaborn-whitegrid')
    plt.xlabel('Nine Most Common Words Used')
    plt.ylabel('Word Quantity')
    plt.show()
word_distribution_vizual()

In [ ]:
def all_words_bigram_wordcloud():
    all_D = pd.Series(str(all_text).split())
    top_20_all_words_bigrams = (pd.Series(nltk.ngrams(all_D, 2))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_all_words_bigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 2 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def all_words_bigram_barplot():
    all_D = pd.Series(str(all_text).split())
    top_20_all_words_bigrams = (pd.Series(nltk.ngrams(all_D, 2))
                          .value_counts()
                          .head(20))
    # Top 3 Words Used in README(s)
    top_20_all_words_bigrams.plot.bar()
    plt.title('Top 2 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()
    
all_words_bigram_wordcloud()
all_words_bigram_barplot()

In [ ]:
def all_words_trigram_wordcloud():
    all_D = pd.Series(str(all_text).split())
    top_20_all_words_trigrams = (pd.Series(nltk.ngrams(all_D, 3))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_all_words_trigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 3 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def all_words_trigram_barplot():
    all_D = pd.Series(str(all_text).split())
    top_20_all_words_trigrams = (pd.Series(nltk.ngrams(all_D, 3))
                          .value_counts()
                          .head(20))
    # Top 3 Words Used in README(s)
    top_20_all_words_trigrams.plot.bar()
    plt.title('Top 3 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()
    
all_words_trigram_wordcloud()
all_words_trigram_barplot()

<hr style="border-top: 10px groove lightblue; margin-top: 1px; margin-bottom: 1px"></hr>

# `Do words in READMEs vary based on programming languages used?`

**Python**

In [ ]:
def python_bigram_wordcloud():
    python_D = pd.Series(str(python_text).split())
    top_20_python_bigrams = (pd.Series(nltk.ngrams(python_D, 2))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_python_bigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 2 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def python_bigram_barplot():
    python_D = pd.Series(str(python_text).split())
    top_20_python_bigrams = (pd.Series(nltk.ngrams(python_D, 2))
                          .value_counts()
                          .head(20))
    # Top 3 Words Used in README(s)
    top_20_python_bigrams.plot.bar()
    plt.title('Top 2 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()
        
python_bigram_wordcloud()
python_bigram_barplot()

In [ ]:
def python_trigram_wordcloud():
    python_D = pd.Series(str(python_text).split())
    top_20_python_trigrams = (pd.Series(nltk.ngrams(python_D, 3))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_python_trigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 3 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()


def python_trigram_barplot():
    python_D = pd.Series(str(python_text).split())
    top_20_python_trigrams = (pd.Series(nltk.ngrams(python_D, 3))
                          .value_counts()
                          .head(20))
    # Top 3 Words Used in README(s)
    top_20_python_trigrams.plot.bar()
    plt.title('Top 3 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()
python_trigram_wordcloud()
python_trigram_barplot()

***
**JavaScript**
- Bigrams

In [ ]:
def javascript_bigram_wordcloud():
    javascript_D = pd.Series(str(javascript_text).split())
    top_20_javascript_bigrams = (pd.Series(nltk.ngrams(javascript_D, 2))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_javascript_bigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 2 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def javascript_bigram_barplot():
    javascript_D = pd.Series(str(javascript_text).split())
    top_20_javascript_bigrams = (pd.Series(nltk.ngrams(javascript_D, 2))
                          .value_counts()
                          .head(20))
    # Top 2 Words Used in README(s)
    top_20_javascript_bigrams.plot.bar()
    plt.title('Top 2 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()
    
javascript_bigram_wordcloud()
javascript_bigram_barplot()

**JavaScript**
- trigrams

In [ ]:
def javascript_trigram_wordcloud():
    javascript_D = pd.Series(str(javascript_text).split())
    top_20_javascript_trigrams = (pd.Series(nltk.ngrams(javascript_D, 3))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_javascript_trigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 3 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def javascript_trigram_barplot():
    javascript_D = pd.Series(str(javascript_text).split())
    top_20_javascript_trigrams = (pd.Series(nltk.ngrams(javascript_D, 3))
                          .value_counts()
                          .head(20))
    # Top 3 Words Used in README(s)
    top_20_javascript_trigrams.plot.bar()
    plt.title('Top 3 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()

javascript_trigram_wordcloud()
javascript_trigram_barplot()

***
**TypeScript**
- Bigrams

In [ ]:
def typescript_bigram_wordcloud():
    typescript_D = pd.Series(str(typescript_text).split())
    top_20_typescript_bigrams = (pd.Series(nltk.ngrams(typescript_D, 2))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_typescript_bigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 2 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def typescript_bigram_barplot():
    typescript_D = pd.Series(str(typescript_text).split())
    top_20_typescript_bigrams = (pd.Series(nltk.ngrams(typescript_D, 2))
                          .value_counts()
                          .head(20))
    # Top 3 Words Used in README(s)
    top_20_typescript_bigrams.plot.bar()
    plt.title('Top 2 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()

typescript_bigram_wordcloud()
typescript_bigram_barplot()

**TypeScript**
- trigrams

In [ ]:
def typescript_trigram_wordcloud():
    typescript_D = pd.Series(str(typescript_text).split())
    top_20_typescript_trigrams = (pd.Series(nltk.ngrams(typescript_D, 3))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_typescript_trigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 3 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def typewscript_trigram_barplot():
    typescript_D = pd.Series(str(typescript_text).split())
    top_20_typescript_trigrams = (pd.Series(nltk.ngrams(typescript_D, 3))
                          .value_counts()
                          .head(20))
    # Top 3 Words Used in README(s)
    top_20_typescript_trigrams.plot.bar()
    plt.title('Top 3 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()

typescript_trigram_wordcloud()
typewscript_trigram_barplot()

***
**Java**
- Bigrams

In [ ]:
def java_bigram_wordcloud():
    java_D = pd.Series(str(java_text).split())
    top_20_java_bigrams = (pd.Series(nltk.ngrams(java_D, 2))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_java_bigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 2 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()
def java_bigram_barplot():
    java_D = pd.Series(str(java_text).split())
    top_20_java_bigrams = (pd.Series(nltk.ngrams(java_D, 2))
                          .value_counts()
                          .head(20))
    # Top 2 Words Used in README(s)
    top_20_java_bigrams.plot.bar()
    plt.title('Top 2 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()
java_bigram_wordcloud()
java_bigram_barplot()

**Java**
- Trigrams

In [ ]:
def java_trigram_wordcloud():
    java_D = pd.Series(str(java_text).split())
    top_20_java_trigrams = (pd.Series(nltk.ngrams(java_D, 3))
                                  .value_counts()
                                  .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_java_trigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 3 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()
def java_trigram_barplot():
    java_D = pd.Series(str(java_text).split())
    top_20_java_trigrams = (pd.Series(nltk.ngrams(java_D, 3))
                                  .value_counts()
                                  .head(20))
    # Top 3 Words Used in README(s)
    top_20_java_trigrams.plot.bar()
    plt.title('Top 3 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()
java_trigram_wordcloud()
java_trigram_barplot()    

***
**Go**
- Bigrams

In [ ]:
def go_bigram_wordcloud():
    go_D = pd.Series(str(go_text).split())
    top_20_go_bigrams = (pd.Series(nltk.ngrams(go_D, 2))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_go_bigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 2 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()
def go_bigram_barplot():
    go_D = pd.Series(str(go_text).split())
    top_20_go_bigrams = (pd.Series(nltk.ngrams(go_D, 2))
                          .value_counts()
                          .head(20))
    # Top 2 Words Used in README(s)
    top_20_go_bigrams.plot.bar()
    plt.title('Top 2 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()
    
go_bigram_wordcloud()
go_bigram_barplot()

**Go**
- Trigrams

In [ ]:
def go_trigram_wordcloud():
    go_D = pd.Series(str(go_text).split())
    top_20_go_trigrams = (pd.Series(nltk.ngrams(go_D, 3))
                          .value_counts()
                          .head(20))
    data = {k[0] + ' ' + k[1]: v for k, v in top_20_go_trigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.title('Top 3 Word clusters Used')
    plt.imshow(img)
    plt.axis('off')
    plt.show()
def go_trigram_barplot():
    go_D = pd.Series(str(go_text).split())
    top_20_go_trigrams = (pd.Series(nltk.ngrams(go_D, 3))
                          .value_counts()
                          .head(20))
    # Top 3 Words Used in README(s)
    top_20_go_trigrams.plot.bar()
    plt.title('Top 3 Word clusters Used')
    plt.xlabel('words')
    plt.ylabel('')
    plt.show()
go_trigram_wordcloud()
go_trigram_barplot()

***
**UNIQUE VALUE COUNTS**

In [ ]:
def unique_word_count():
    # Visual of the unique word counts
    train['unique_word_counts'] = train.lemmatized.apply(lambda x : len(set(x.split())))

    unique_train = pd.DataFrame(train.groupby('language').unique_word_counts.mean().sort_values(ascending=False)).reset_index()
    unique_train.head()

    ax = sns.barplot(x='language',y='unique_word_counts',data=unique_train)
    ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
    plt.show()
unique_word_count()

In [ ]:
def min_words_in_read_mes():
    # min README(s)
    train.lemmatized.apply(len).sort_values().head(20).plot.bar(x=train.repo)
    plt.title('Bottom 20 Shortest README Files')
    plt.xlabel('Repository (Index Number)')
    plt.ylabel('')
    plt.show()
min_words_in_read_mes()

<hr style="border-top: 10px groove lightblue; margin-top: 1px; margin-bottom: 1px"></hr>

# `Do lenght of words vary in READMEs based on programming languages?`